<a href="https://colab.research.google.com/github/quanndm/kltn/blob/dev/notebooks/tumor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Init - import library - download data

In [1]:
!rm -rf kltn
!rm -rf sample_data
!git clone -b dev https://github.com/quanndm/kltn.git

Cloning into 'kltn'...
remote: Enumerating objects: 1224, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 1224 (delta 3), reused 8 (delta 2), pack-reused 1212 (from 2)
Receiving objects: 100% (1224/1224), 13.28 MiB | 14.16 MiB/s, done.
Resolving deltas: 100% (791/791), done.


In [2]:
from kltn.init.install_dependencies import install_packages, load_config
install_packages("./kltn")
config = load_config("./kltn/parameters.yaml")

In [3]:
import torch
import matplotlib.pyplot as plt
import numpy as np
from google.colab import drive

from kltn import *
from kltn.datasets.get_datasets import get_datasets_stage2
from kltn.utils.logger import init_logger
from kltn.utils.train_val_epochs import trainer, trainer_stage2
from kltn.utils.metrics import  DiceMetric, TverskyLossWSigmoid
from kltn.models.model_factory import ModelFactory
from kltn.utils.visualize_results import visualize_results

In [4]:
drive.flush_and_unmount()
drive.mount("/content/drive")

Mounted at /content/drive


In [5]:
LOGGER = init_logger(config["log_path"])
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [6]:
from kltn.datasets.prepare_data import prepare_dataset_LiTS
prepare_dataset_LiTS(dir_name=config["source_folder_lits"])

0.00% complete (down: 0.0 kB/s up: 0.0 kB/s peers: 1) downloading LITS17 (https://academictorrents.com/announce.php)[127.0.0.1:6881] v1 skipping tracker announce (unreachable) "" (1)
LITS17 (https://ipv6.academictorrents.com/announce.php)[127.0.0.1:6881] v1 skipping tracker announce (unreachable) "" (1)
LITS17 (https://academictorrents.com/announce.php)[172.28.0.12:6881] v1 unspecified system error "" (1)
LITS17 (https://ipv6.academictorrents.com/announce.php)[172.28.0.12:6881] v1 unspecified system error "" (1)
LITS17 (udp://tracker.opentrackr.org:1337/announce)[127.0.0.1:6881] v1 skipping tracker announce (unreachable) "" (1)
2.12% complete (down: 31139.7 kB/s up: 928.0 kB/s peers: 10) downloading LITS17 (https://academictorrents.com/announce.php)[127.0.0.1:6881] v1 skipping tracker announce (unreachable) "" (2)
LITS17 (https://ipv6.academictorrents.com/announce.php)[127.0.0.1:6881] v1 skipping tracker announce (unreachable) "" (2)
LITS17 (https://academictorrents.com/announce.php)[1

## Model

In [6]:
weight_path = "/content/drive/MyDrive/KLTN/code/weight/best_metric_model_UNet3DWResNeXtCoT.pth"
model_stage_1 = ModelFactory.get_model("unet3d_resnextcot",in_channels=1, n_classes=3, n_channels=16).to(device)
model_stage_1.load_state_dict(torch.load(weight_path, map_location=device))

<All keys matched successfully>

In [7]:
model = ModelFactory.get_model("unet3d_resnextcot",in_channels=1, n_classes=1, n_channels=16).to(device)
model.load_state_dict({
    k: v for k, v in model_stage_1.state_dict().items() if "out" not in k
}, strict=False)

_IncompatibleKeys(missing_keys=['out.conv.weight', 'out.conv.bias'], unexpected_keys=[])

## Load data

In [8]:
full_train_dataset, val_dataset = get_datasets_stage2(source_folder=config["source_folder_lits"], seed=123, fold_number=2, model_stage_1=model_stage_1)
print(len(full_train_dataset), len(val_dataset))

105 26


In [9]:
train_loader = torch.utils.data.DataLoader(full_train_dataset, batch_size=1, shuffle=True,
                                           num_workers=0, pin_memory=True, drop_last=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=1, shuffle=False,
                                         pin_memory=True, num_workers=0)

## Training

In [10]:
criterion = TverskyLossWSigmoid().to(device)

dice_acc = DiceMetric(include_background=False, reduction='mean_batch', get_not_nans=True)

optimizer = torch.optim.AdamW(model.parameters(), lr=float(config["learning_rate_train"]), weight_decay=float(config["weight_decay"]))
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=int(config["max_epochs"]), eta_min = float(config["eta_min"]))

In [11]:
LOGGER.info("[TRAINER] Start TRAIN process...")

(
    val_acc_max,
    best_epoch,
    dices_tumor,
    loss_epochs,
    trains_epoch,
    ious_tumor,
    precisions_tumor,
    recalls_tumor,
    time_tmp
) = trainer_stage2(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    optimizer=optimizer,
    loss_func=criterion,
    acc_func=dice_acc,
    scheduler=scheduler,
    batch_size=config["batch_size"],
    max_epochs = 10,
    start_epoch = config["start_epoch"],
    val_every=config["val_every"],
    path_save_model=config["path_save_model_state"],
    logger=LOGGER,
    save_model=False,
    post_fix="tumor"
)

[TRAINER] Start TRAIN process...
INFO:kltn.utils.logger:[TRAINER] Start TRAIN process...

==============================Training epoch 1==============================
INFO:kltn.utils.logger:
==============================Training epoch 1==============================


Epoch 1/10 1/105 loss: 0.8004 time 3.37s
Epoch 1/10 2/105 loss: 0.7922 time 6.62s
Epoch 1/10 3/105 loss: 0.7786 time 3.48s
Epoch 1/10 4/105 loss: 0.7712 time 5.17s
Epoch 1/10 5/105 loss: 0.7732 time 1.77s
Epoch 1/10 6/105 loss: 0.7686 time 5.82s
Epoch 1/10 7/105 loss: 0.7637 time 1.39s
Epoch 1/10 8/105 loss: 0.7617 time 6.27s
Epoch 1/10 9/105 loss: 0.7643 time 2.92s
Epoch 1/10 10/105 loss: 0.7654 time 6.07s
Epoch 1/10 11/105 loss: 0.7659 time 1.84s
Epoch 1/10 12/105 loss: 0.7663 time 1.18s
Epoch 1/10 13/105 loss: 0.7670 time 6.08s
Epoch 1/10 14/105 loss: 0.7679 time 5.82s
Epoch 1/10 15/105 loss: 0.7681 time 2.57s
Epoch 1/10 16/105 loss: 0.7659 time 9.37s
Epoch 1/10 17/105 loss: 0.7663 time 1.39s
Epoch 1/10 18/105 loss: 0.7672 time 4.67s
Epoch 1/10 19/105 loss: 0.7675 time 2.22s
Epoch 1/10 20/105 loss: 0.7674 time 2.55s
Epoch 1/10 21/105 loss: 0.7677 time 6.87s
Epoch 1/10 22/105 loss: 0.7681 time 4.47s
Epoch 1/10 23/105 loss: 0.7684 time 4.60s
Epoch 1/10 24/105 loss: 0.7683 time 5.78s
E

Final training epochs: 1/10 ---[loss: 0.7523] ---[time 431.36s]
INFO:kltn.utils.logger:Final training epochs: 1/10 ---[loss: 0.7523] ---[time 431.36s]

********************Epoch 1 Validation********************
INFO:kltn.utils.logger:
********************Epoch 1 Validation********************


Epoch 1/10 105/105 loss: 0.7523 time 5.86s


/content/kltn/utils/metrics.py:151: RuntimeWarning: invalid value encountered in divide
  self.avg = self.sum / self.count


IndexError: list index out of range

In [ ]:
import json
path_save_result = f"/content/drive/MyDrive/KLTN/code/result_model_{model.__class__.__name__}_tumor.json"

results = {
    "dice_tumor": [float(x) for x in np.array(dices_tumor, dtype=np.float32)],
    "loss": [float(x) for x in np.array(loss_epochs, dtype= np.float32)],
    "iou_tumor": [float(x) for x in np.array(ious_tumor, dtype=np.float32)],
    "precision_tumor": [float(x) for x in np.array(precisions_tumor, dtype=np.float32)],
    "recall_tumor": [float(x) for x in np.array(recalls_tumor, dtype=np.float32)],
    "best_epoch": best_epoch,
    "time_train": time_tmp
}
with open(path_save_result, "w") as f:
    json.dump(results, f, indent=4)